# 仓库粒度的冲突消解-数据集预处理

## 1.top4 数量的仓库冲突内容提取

In [7]:
import json

# 读取文件中的JSON数据
input_path = "G:/now/2024merge/mergeMinePython/json/50Repo/50RepoV1Pretty.json"
with open(input_path, 'r', encoding='utf-8') as file:
    data_list = json.load(file)

# 打开目标txt文件以写入内容
with open('dataset/repoLevel/XCoLab_Conflicts.txt', 'w', encoding='utf-8') as output_file:
    for data in data_list:
        if data.get('file_name') == "./index_conflict_files/XCoLab/":
            a_contents = data.get('a_contents', '')
            o_contents = data.get('o_contents', '')
            b_contents = data.get('b_contents', '')
            conflict = 'concurrent_change\n' + a_contents + '\nbase_content\n' + o_contents + '\nincoming_change\n' + b_contents
            processed_content = conflict.replace('\n', ' _newline_ ').replace('\t', ' ')
            output_file.write(processed_content + '\n')

print("处理完毕，结果已写入到 XCoLab_Conflicts.txt 文件中。")


处理完毕，结果已写入到 XCoLab_Conflicts.txt 文件中。


In [8]:
with open(input_path, 'r', encoding='utf-8') as file:
    data_list = json.load(file)

# 打开目标txt文件以写入内容
with open('dataset/repoLevel/accumulo_Conflicts.txt', 'w', encoding='utf-8') as output_file:
    for data in data_list:
        if data.get('file_name') == "./index_conflict_files/accumulo/":
            a_contents = data.get('a_contents', '')
            o_contents = data.get('o_contents', '')
            b_contents = data.get('b_contents', '')
            conflict = 'concurrent_change\n' + a_contents + '\nbase_content\n' + o_contents + '\nincoming_change\n' + b_contents
            processed_content = conflict.replace('\n', ' _newline_ ').replace('\t', ' ')
            output_file.write(processed_content + '\n')

print("处理完毕，结果已写入到 accumulo.txt 文件中。")


处理完毕，结果已写入到 accumulo.txt 文件中。


In [9]:
with open(input_path, 'r', encoding='utf-8') as file:
    data_list = json.load(file)

# 打开目标txt文件以写入内容
with open('dataset/repoLevel/elasticsearch_Conflicts.txt', 'w', encoding='utf-8') as output_file:
    for data in data_list:
        if data.get('file_name') == "./index_conflict_files/elasticsearch/":
            a_contents = data.get('a_contents', '')
            o_contents = data.get('o_contents', '')
            b_contents = data.get('b_contents', '')
            conflict = 'concurrent_change\n' + a_contents + '\nbase_content\n' + o_contents + '\nincoming_change\n' + b_contents
            processed_content = conflict.replace('\n', ' _newline_ ').replace('\t', ' ')
            output_file.write(processed_content + '\n')

print("处理完毕，结果已写入到 elasticsearch.txt 文件中。")


处理完毕，结果已写入到 elasticsearch.txt 文件中。


In [10]:
with open(input_path, 'r', encoding='utf-8') as file:
    data_list = json.load(file)

# 打开目标txt文件以写入内容
with open('dataset/repoLevel/OpenFaces_Conflicts.txt', 'w', encoding='utf-8') as output_file:
    for data in data_list:
        if data.get('file_name') == "./index_conflict_files/OpenFaces/":
            a_contents = data.get('a_contents', '')
            o_contents = data.get('o_contents', '')
            b_contents = data.get('b_contents', '')
            conflict = 'concurrent_change\n' + a_contents + '\nbase_content\n' + o_contents + '\nincoming_change\n' + b_contents
            processed_content = conflict.replace('\n', ' _newline_ ').replace('\t', ' ')
            output_file.write(processed_content + '\n')

print("处理完毕，结果已写入到 OpenFaces.txt 文件中。")


处理完毕，结果已写入到 OpenFaces.txt 文件中。


## 2.代码块切分

## 3.方法签名 -> 信号量

In [12]:
import ast
import hashlib
import os

# Define a normalized node type (using strings for simplicity)
class NormalizedNode:
    def __init__(self, kind, value=None):
        self.kind = kind
        self.value = value
    def __repr__(self):
      if self.value:
          return f"({self.kind}, {self.value})"
      else:
          return f"({self.kind})"

# Simple AST Normalizer class
class AstNormalizer(ast.NodeVisitor):
    def __init__(self):
        self.normalized_ast = []
        self.variable_mapping = {}
        self.variable_counter = 0

    def process_ident(self, id_node):
        # Change here: Get the name from the appropriate attribute
        name = id_node.id if isinstance(id_node, ast.Name) else id_node.arg
        if name in self.variable_mapping:
            canonical_name = self.variable_mapping[name]
            self.normalized_ast.append(NormalizedNode("ident", canonical_name))
        else:
            canonical_name = f"var{self.variable_counter}"
            self.variable_mapping[name] = canonical_name
            self.normalized_ast.append(NormalizedNode("ident", canonical_name))
            self.variable_counter += 1


    def visit_FunctionDef(self, node):
      self.normalized_ast.append(NormalizedNode("keyword", "function"))
      self.generic_visit(node)

    def visit_Name(self, node):
      self.process_ident(node)

    def visit_arg(self, node):
      self.process_ident(node)

    def visit_stmt(self, node):
       self.normalized_ast.append(NormalizedNode("statement", ast.unparse(node)))
       self.generic_visit(node)

    def visit_expr(self, node):
       self.normalized_ast.append(NormalizedNode("expression", ast.unparse(node)))
       self.generic_visit(node)

    def visit_block(self, node):
      self.normalized_ast.append(NormalizedNode("block", ast.unparse(node)))
      self.generic_visit(node)


# Function to normalize an AST
def normalize_ast(code):
    tree = ast.parse(code)
    normalizer = AstNormalizer()
    normalizer.visit(tree)
    return normalizer.normalized_ast


# Simple hash function for a normalized AST
def hash_normalized_ast(normalized_ast):
    hashable_string = str(normalized_ast) # Convert the list of normalized nodes to a string for hashing
    return hashlib.sha256(hashable_string.encode()).hexdigest()


def detect_homologous_functions(code):
    tree = ast.parse(code)
    function_hashes = {}

    for item in tree.body:
        if isinstance(item, ast.FunctionDef):
            normalized_ast = normalize_ast(ast.unparse(item))
            hash_value = hash_normalized_ast(normalized_ast)

            if hash_value not in function_hashes:
                function_hashes[hash_value] = []
            function_hashes[hash_value].append(item.name)

    print("Potential homologous functions:")
    for hash_val, functions in function_hashes.items():
        if len(functions) > 1:
            print(f"   Hash: {hash_val} Functions: {functions}")


def process_directory(directory_path):
    if os.path.isdir(directory_path):
        for entry in os.listdir(directory_path):
            file_path = os.path.join(directory_path, entry)
            if os.path.isfile(file_path):
                print(f"Analyzing file: {file_path}")
                with open(file_path, "r") as f:
                  code = f.read()
                  detect_homologous_functions(code)

    elif os.path.isfile(directory_path):
            print(f"Analyzing file: {directory_path}")
            with open(directory_path, "r") as f:
                code = f.read()
                detect_homologous_functions(code)
    else:
        print(f"Invalid file or directory: {directory_path}")

# Example usage
if __name__ == "__main__":
    # Create an example.py file in the current directory for testing
    example_code = """
def add(a, b):
    sum = a + b
    return sum

def sub(c, d):
    diff = c - d
    return diff

def sum_all(a, b):
    result = a + b
    return result

def sum_all_again(x, y):
    z = x + y
    return z
"""
    with open("example.py", "w") as f:
        f.write(example_code)
    process_directory("example.py")
    # To test on a directory containing several .py files, replace the path by the directory path in the process_directory method

Analyzing file: example.py
Potential homologous functions:
   Hash: f4f0b970f9afa6868a8670e1ccaf8229bcfcff7f57870d7e2383ce531b058898 Functions: ['add', 'sub', 'sum_all', 'sum_all_again']
